In [1]:
import os
import shutil
import glob
import wandb

In [2]:
sweep_name="SWEEPComplete"
root_path="running/sweepcomplete"
output_from="output/"
output_to_path="output_sweep_complete/"
checkpoint_order={
    "cifar10":0,
    "cifar100":1,
    "grocerystore":2,
    "svhn":3,
    "tinyimagenet":4
}

In [3]:
def get_max_str(lst):
    return max(lst, key=len)

In [4]:
def find_identifiers(run_name):
    paths=glob.glob(os.path.join(root_path,f"lowres_*/*{run_name}"))    
    if len(paths)!=1:
        raise FileNotFoundError(f"Run {run_name} could not be found")
    glob_tokens=paths[0].split("/")
    path_tokens=get_max_str(glob_tokens).split("_")
    run_tokens=glob_tokens[3].split(f"_{run_name}")[0]
    run=run_tokens.split(f"_{run_name}")[0]
    if len(path_tokens)==4:
        model=f"{path_tokens[0]}_{path_tokens[1]}_{path_tokens[2]}"
        dataset=path_tokens[3].split(sweep_name)[1]
    else:
        model=f"{path_tokens[0]}_{path_tokens[1]}"
        dataset=path_tokens[2].split(sweep_name)[1]    
    return model,dataset,run

In [5]:
def copy_sweep(run_name):
    model,dataset,run=find_identifiers(run_name)
    from_path=os.path.join(root_path,output_from,dataset,model,f"version_{run}")
    to_path=os.path.join(output_to_path,dataset,model,f"version_{run}")
    from_path_checkpoint=glob.glob(os.path.join(root_path,f"lowres_*/*{run_name}/checkpoints"))[0]
    to_path_checkpoint=os.path.join(output_to_path,dataset,model,f"version_{run}/checkpoints")
    
    appendix=2
    while(os.path.exists(to_path)):
        to_path=os.path.join(output_to_path,dataset,model,f"version_{run}_{appendix}")
        to_path_checkpoint=os.path.join(output_to_path,dataset,model,f"version_{run}_{appendix}/checkpoints")
        appendix=appendix+1
    shutil.copytree(from_path,to_path)
    shutil.copytree(from_path_checkpoint,to_path_checkpoint)
    return run, dataset

In [6]:
def get_best_run(sweep_id):
    api = wandb.Api()
    sweep = api.sweep(sweep_id)       
    model, dataset, _id=sweep.best_run(order="summary_metrics.acc_max/val").name.split("-")
    dataset_trained_on=str(sweep.name).replace(model,"")
    return model, dataset_trained_on, _id

In [7]:
def get_sweeps(command_path):
    with open(command_path,"r") as file:
        lines = file.readlines()
        sweep_list = [line.rstrip().split(" ")[2] for line in lines]
    return sweep_list

In [8]:
def copy_sweeps(sweep_command_path):
    sweeps=get_sweeps(sweep_command_path)
    for sweep in sweeps:
        try:
            model,dataset_trained_on,best_run_id=get_best_run(sweep)
            # print(dataset_trained_on)
            run_id,dataset=copy_sweep(best_run_id)            
            os.rename(os.path.join(output_to_path,dataset,model,f"version_{run_id}"),os.path.join(output_to_path,dataset,model,f"version_{checkpoint_order[dataset_trained_on]}"))
        except:
            print("could not copy")
            
        # print(os.path.join(output_to_path,dataset,model,f"version_{run_id}"),os.path.join(output_to_path,dataset,model,f"version_{checkpoint_order[dataset_trained_on]}"))

In [15]:
copy_sweeps("running/sweepcomplete/python_files_sweep/SWEEPCompletetinyimagenet/sweep_agent_commands17_07_2022_11.txt")

wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorting runs by -summary_metrics.acc_max/val
wandb: Sorti

In [10]:
# copy_sweep("sirlngcd")